Reinforcement Learning Tictactoe:

An attempt to build an agent which learns how to play the three by three tictactoe game, using reinforcement learning. In particular, this is my implementation of the most basic concepts discussed in the first chapter of Sutton and Barto's <i>Introduction to Reinforcement Learning</i>.

In [1]:
import numpy as np
import pandas as pd

The data consists of ten columns, the first nine corresponding to one position on the board and the last determining whether the game is a win or not for player 'x'.

In [2]:
data=pd.read_csv('endgame.csv',header=None)

print(data.head())

   0  1  2  3  4  5  6  7  8         9
0  x  x  x  x  o  o  x  o  o  positive
1  x  x  x  x  o  o  o  x  o  positive
2  x  x  x  x  o  o  o  o  x  positive
3  x  x  x  x  o  o  o  b  b  positive
4  x  x  x  x  o  o  b  o  b  positive


In [3]:
A=1*(data[:][9].values=='positive')

A=A.reshape((data.shape[0],1))
np.shape(A)

(958, 1)

In [4]:
def checkwin(A,i,player):
    f=0
    if A[i][0]==A[i][1] and A[i][1]==A[i][2] and A[i][0]==player:     #0 1 2
        f=1
    elif (A[i][3]==A[i][4] and A[i][4]==A[i][5] and A[i][3]==player): #3 4 5
        f=1
    elif A[i][6]==A[i][7] and A[i][7]==A[i][8] and A[i][6]==player:   #6 7 8
        f=1
    elif A[i][0]==A[i][3] and A[i][3]==A[i][6] and A[i][0]==player:   #0 3 6
        f=1
    elif A[i][1]==A[i][4] and A[i][4]==A[i][7] and A[i][1]==player:   #1 4 7
        f=1
    elif A[i][2]==A[i][5] and A[i][5]==A[i][8] and A[i][2]==player:   #2 5 8
        f=1
    elif A[i][0]==A[i][4] and A[i][4]==A[i][8] and A[i][0]==player:   #0 4 8
        f=1
    elif A[i][2]==A[i][4] and A[i][4]==A[i][6] and A[i][2]==player:   #2 4 6
        f=1
    
    return f

In [5]:
lose=0
draw=0
for i in range(A.shape[0]):
    if A[i][0]!=1:
        if checkwin(data.values,i,'o')==0:
            A[i][0]=0.5
            draw=draw+1
        else:
            A[i][0]=0
            lose=lose+1
            
print("Games lost: {}\nGames drawed: {}".format(lose,draw))

Games lost: 316
Games drawed: 16


In [11]:
state_table=np.zeros(9)
for i in range(9):
    x=1*(data[:][i].values=='x')
    x=x.reshape((data.shape[0],1))
    state_table[i]=np.sum(x*A)/np.sum(x)
    print(i," ",state_table[i])

0   0.7057416267942583
1   0.5952380952380952
2   0.7057416267942583
3   0.5952380952380952
4   0.7991266375545851
5   0.5952380952380952
6   0.7057416267942583
7   0.5952380952380952
8   0.7057416267942583


In [7]:
ind=np.arange(9)
result = np.where(state_table[ind] == np.amax(state_table[ind]))
print(int(result[0]))

4


As we can see, index 4, or the center piece, has the highest probability of winning for x.

In [12]:
temp=np.zeros_like(state_table)
temp=temp+state_table
print(temp)

[0.70574163 0.5952381  0.70574163 0.5952381  0.79912664 0.5952381
 0.70574163 0.5952381  0.70574163]


In [13]:
#single game
ind=np.arange(9)

alpha=0.1
game=np.array([['b','b','b','b','b','b','b','b','b']]) #blank board
prev=0
for i in range(9):
    if i%2==0: #player x
        move = np.where(state_table[ind] == np.amax(state_table[ind]))
        if i==0:
            prev=ind[move[0][0]]
        else:
            cur=ind[move[0][0]]
            state_table[prev]=state_table[prev]+alpha*(state_table[cur]-state_table[prev])
            prev=cur
        
        game[0][ind[move[0][0]]]='x'
        ind=np.delete(ind,move[0][0])
        if checkwin(game,0,'x')==1:
            break
    else: #player o
        move = np.where(state_table[ind] == np.amin(state_table[ind]))
        if i==1:
            prev=ind[move[0][0]]
        else:
            cur=ind[move[0][0]]
            state_table[prev]=state_table[prev]+alpha*(state_table[cur]-state_table[prev])
            prev=cur
        game[0][ind[move[0][0]]]='o'
        ind=np.delete(ind,move[0][0])
        if checkwin(game,0,'o')==1:
            break
    print('Turn ',i+1)
    print(game[0][0:3])
    print(game[0][3:6])
    print(game[0][6:9])
    print('\n')
        

Turn  1
['b' 'b' 'b']
['b' 'x' 'b']
['b' 'b' 'b']


Turn  2
['b' 'o' 'b']
['b' 'x' 'b']
['b' 'b' 'b']


Turn  3
['x' 'o' 'b']
['b' 'x' 'b']
['b' 'b' 'b']


Turn  4
['x' 'o' 'b']
['o' 'x' 'b']
['b' 'b' 'b']


Turn  5
['x' 'o' 'x']
['o' 'x' 'b']
['b' 'b' 'b']


Turn  6
['x' 'o' 'x']
['o' 'x' 'o']
['b' 'b' 'b']




In [16]:
print(temp)
print(state_table)
print('\nDifference\n',temp-state_table)

[0.70574163 0.5952381  0.70574163 0.5952381  0.79912664 0.5952381
 0.70574163 0.5952381  0.70574163]
[0.69469127 0.60628845 0.69469127 0.60628845 0.79912664 0.60628845
 0.70574163 0.5952381  0.70574163]

Difference
 [ 0.01105035 -0.01105035  0.01105035 -0.01105035  0.         -0.01105035
  0.          0.          0.        ]
